Queremos fazer uma análise de quais partidos,
mais especificamente governos de UFs que foram mais permissivos 
em relação à degradação do meio ambiente.
Para isso, precisamos de dados do meio ambiente e de eleições.

<h3>Instalando base dos dados

In [3]:
!pip install basedosdados

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


Importando ferramentas

In [ ]:
import basedosdados as bd
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go

<h3>Lendo tabela de informação de candidatos eleitos

<h4> Para pegar as informações relevantes, selecionamos só os governantes de UFs que têm dados sobre sua cobertura de mata amazônia na basedosdados

In [31]:
csvGov= pd.read_csv('ConsultasEmCsv/governosestados.csv')

In [32]:
csvGov

,ano,sigla_uf,sigla_partido
0,1994,AC,PPR
1,1998,AC,PT
2,2002,AC,PT
3,2006,AC,PT
4,2010,AC,PT
...,...,...,...
56,2006,TO,PMDB
57,2010,TO,PSDB
58,2014,TO,PHS
59,2014,TO,PMDB


<h3>Lendo tabela de informação de cobertura da mata atlântica por estado e ano

In [35]:
csvCob= pd.read_csv('ConsultasEmCsv/coberturaanoestado.csv')

In [36]:
csvCob

,ano,estado_abrev,areatotal
0,1998,AC,152234.087485
1,1999,AC,151839.913006
2,2000,AC,151357.782425
3,2001,AC,151087.733573
4,2002,AC,148017.291313
...,...,...,...
193,2015,TO,6951.741695
194,2016,TO,6592.706262
195,2017,TO,6853.481358
196,2018,TO,6645.391944


In [37]:
lig = pd.read_csv('ConsultasEmCsv/coberturajoinpartidos.csv')

In [45]:
lig

,ano,sigla_uf,areatotal,sigla_partido
0,1998,AC,1.522341e+05,PT
1,2002,AC,1.480173e+05,PT
2,2006,AC,1.477197e+05,PT
3,2010,AC,1.465175e+05,PT
4,2014,AC,1.456493e+05,PT
5,2018,AC,1.441178e+05,PP
6,1998,AM,1.503607e+06,PFL
7,2002,AM,1.503723e+06,PPS
8,2006,AM,1.497690e+06,PMDB
9,2010,AM,1.496009e+06,PMN


In [46]:
def sub(x,y):
    return x- y

In [66]:
lig["delta"] = None
lig

,ano,sigla_uf,areatotal,sigla_partido,delta
0,1998,AC,1.522341e+05,PT,None
1,2002,AC,1.480173e+05,PT,None
2,2006,AC,1.477197e+05,PT,None
3,2010,AC,1.465175e+05,PT,None
4,2014,AC,1.456493e+05,PT,None
5,2018,AC,1.441178e+05,PP,None
6,1998,AM,1.503607e+06,PFL,None
7,2002,AM,1.503723e+06,PPS,None
8,2006,AM,1.497690e+06,PMDB,None
9,2010,AM,1.496009e+06,PMN,None


In [83]:
lig["delta"] = None
for index in lig.index:
    line = lig.iloc[index]
    ano = line.ano
    est = line.sigla_uf
    areaantiga = line.areatotal
    areaatual = lig[(lig.sigla_uf==est) & (lig.ano == ano + 4)].areatotal
    if len(areaatual) == 0:
        areaatual= None
    else:
        areaatual= areaatual.iloc[0]
        lig.loc[index,'delta'] = ((areaatual - areaantiga)/ areaantiga)
   
lig

,ano,sigla_uf,areatotal,sigla_partido,delta
0,1998,AC,1.522341e+05,PT,-0.027699
1,2002,AC,1.480173e+05,PT,-0.00201
2,2006,AC,1.477197e+05,PT,-0.008139
3,2010,AC,1.465175e+05,PT,-0.005925
4,2014,AC,1.456493e+05,PT,-0.010515
5,2018,AC,1.441178e+05,PP,None
6,1998,AM,1.503607e+06,PFL,0.000077
7,2002,AM,1.503723e+06,PPS,-0.004012
8,2006,AM,1.497690e+06,PMDB,-0.001123
9,2010,AM,1.496009e+06,PMN,-0.002121
